In [1]:
import pandas as pd
import bioframe as bf

In [2]:
data_path = '/scratch/groups/horence/angelika/EM_expression/data/'

In [3]:
genecode_path = data_path+'gencode.v38.annotation.gff3.gz'

In [4]:
gencode = bf.read_table(genecode_path, schema='gff', skiprows=7)

In [5]:
genes = gencode.loc[gencode['feature'].values=='gene']
genes = genes.astype({'start':int,'end':int})
### need to join gene table & expression table
geneNames =  genes['attributes'].str.split(';')
genes['Description'] = geneNames.str[3].str.split('=').str[1]
genes['Name'] = geneNames.str[0].str.split('=').str[1]
genes.head()

,chrom,source,feature,start,end,score,strand,frame,attributes,Description,Name
0,chr1,HAVANA,gene,11869,14409,.,+,.,ID=ENSG00000223972.5;gene_id=ENSG00000223972.5...,DDX11L1,ENSG00000223972.5
12,chr1,HAVANA,gene,14404,29570,.,-,.,ID=ENSG00000227232.5;gene_id=ENSG00000227232.5...,WASH7P,ENSG00000227232.5
25,chr1,ENSEMBL,gene,17369,17436,.,-,.,ID=ENSG00000278267.1;gene_id=ENSG00000278267.1...,MIR6859-1,ENSG00000278267.1
28,chr1,HAVANA,gene,29554,31109,.,+,.,ID=ENSG00000243485.5;gene_id=ENSG00000243485.5...,MIR1302-2HG,ENSG00000243485.5
36,chr1,ENSEMBL,gene,30366,30503,.,+,.,ID=ENSG00000284332.1;gene_id=ENSG00000284332.1...,MIR1302-2,ENSG00000284332.1


In [6]:
trans_df = gencode[(gencode.feature == 'transcript')].copy()

In [7]:
trans_df['Parent'] = trans_df.attributes.str.split(';').str[1].str.split('=').str[1]
trans_df['tID'] = trans_df.attributes.str.split(';').str[0].str.split('=').str[1]

In [23]:
trans_df

,chrom,source,feature,start,end,score,strand,frame,attributes,Parent,tID
1,chr1,HAVANA,transcript,11869.0,14409.0,.,+,.,ID=ENST00000456328.2;Parent=ENSG00000223972.5;...,ENSG00000223972.5,ENST00000456328.2
5,chr1,HAVANA,transcript,12010.0,13670.0,.,+,.,ID=ENST00000450305.2;Parent=ENSG00000223972.5;...,ENSG00000223972.5,ENST00000450305.2
13,chr1,HAVANA,transcript,14404.0,29570.0,.,-,.,ID=ENST00000488147.1;Parent=ENSG00000227232.5;...,ENSG00000227232.5,ENST00000488147.1
26,chr1,ENSEMBL,transcript,17369.0,17436.0,.,-,.,ID=ENST00000619216.1;Parent=ENSG00000278267.1;...,ENSG00000278267.1,ENST00000619216.1
29,chr1,HAVANA,transcript,29554.0,31097.0,.,+,.,ID=ENST00000473358.1;Parent=ENSG00000243485.5;...,ENSG00000243485.5,ENST00000473358.1
...,...,...,...,...,...,...,...,...,...,...,...
3148142,chrM,ENSEMBL,transcript,14149.0,14673.0,.,-,.,ID=ENST00000361681.2;Parent=ENSG00000198695.2;...,ENSG00000198695.2,ENST00000361681.2
3148147,chrM,ENSEMBL,transcript,14674.0,14742.0,.,-,.,ID=ENST00000387459.1;Parent=ENSG00000210194.1;...,ENSG00000210194.1,ENST00000387459.1
3148150,chrM,ENSEMBL,transcript,14747.0,15887.0,.,+,.,ID=ENST00000361789.2;Parent=ENSG00000198727.2;...,ENSG00000198727.2,ENST00000361789.2
3148155,chrM,ENSEMBL,transcript,15888.0,15953.0,.,+,.,ID=ENST00000387460.2;Parent=ENSG00000210195.2;...,ENSG00000210195.2,ENST00000387460.2


In [8]:
trans_counts = trans_df.groupby('Parent').size().reset_index(name='counts')
trans_counts.head()

,Parent,counts
0,ENSG00000000003.15,5
1,ENSG00000000005.6,2
2,ENSG00000000419.14,16
3,ENSG00000000457.14,5
4,ENSG00000000460.17,9


In [9]:
genes =  pd.merge(genes, trans_counts, left_on='Name', right_on='Parent')

In [27]:
genes_filtered = genes[genes.counts > 5].reset_index(drop=True)

In [32]:
genes_filtered.to_csv('./processed_files/genes_filtered.tsv', sep='\t', index=False)

In [33]:
genes_filtered

,chrom,source,feature,start,end,score,strand,frame,attributes,Description,Name,Parent,counts
0,chr1,HAVANA,gene,141474,173862,.,-,.,ID=ENSG00000241860.7;gene_id=ENSG00000241860.7...,RP11-34P13.13,ENSG00000241860.7,ENSG00000241860.7,6
1,chr1,HAVANA,gene,257864,359681,.,-,.,ID=ENSG00000228463.10;gene_id=ENSG00000228463....,AP006222.1,ENSG00000228463.10,ENSG00000228463.10,6
2,chr1,HAVANA,gene,365389,522928,.,-,.,ID=ENSG00000237094.12;gene_id=ENSG00000237094....,RP4-669L17.4,ENSG00000237094.12,ENSG00000237094.12,25
3,chr1,HAVANA,gene,586071,827796,.,-,.,ID=ENSG00000230021.10;gene_id=ENSG00000230021....,RP11-206L10.17,ENSG00000230021.10,ENSG00000230021.10,15
4,chr1,HAVANA,gene,778747,810065,.,+,.,ID=ENSG00000237491.10;gene_id=ENSG00000237491....,LINC01409,ENSG00000237491.10,ENSG00000237491.10,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12201,chrY,HAVANA,gene,20464916,20575519,.,-,.,ID=ENSG00000229236.3;gene_id=ENSG00000229236.3...,TTTY10,ENSG00000229236.3,ENSG00000229236.3,13
12202,chrY,HAVANA,gene,23219434,23291356,.,+,.,ID=ENSG00000205944.11;gene_id=ENSG00000205944....,DAZ2,ENSG00000205944.11,ENSG00000205944.11,9
12203,chrY,HAVANA,gene,24833843,24907040,.,+,.,ID=ENSG00000205916.12;gene_id=ENSG00000205916....,DAZ4,ENSG00000205916.12,ENSG00000205916.12,8
12204,chrY,HAVANA,gene,57067865,57130289,.,+,.,ID=ENSG00000124333.16_PAR_Y;gene_id=ENSG000001...,VAMP7,ENSG00000124333.16_PAR_Y,ENSG00000124333.16_PAR_Y,6


In [34]:
# exons = gencode[gencode.feature=='exon'].copy()
# exons['Parent'] = exons.attributes.str.split(';').str[1].str.split('=').str[1]
# exons['eID'] = exons.attributes.str.split(';').str[0].str.split('=').str[1]

In [39]:
# exons

In [36]:
# gtex_genes_path = data_path+'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct.gz'
# gtex_junctions_path = data_path+'GTEx_Analysis_2017-06-05_v8_STARv2.5.3a_junctions.gct.gz'

In [37]:
# gtex_junctions = pd.read_csv(gtex_junctions_path, delimiter='\t', skiprows = 2, nrows=1000, )

In [38]:
# gtex_junctions[gtex_junctions.Description=='ENSG00000227232.5']